10/23/24 <br>
Written to convert CSV to desired excel formatting <br>

26.0908 - Exercise Physiology and Kinesiology <br>
31.0505 - Exercise Science and Kinesiology



This script was used to analyze the number and type of Kinesiology programs across the US. UVA proposed a new program and wanted data to validate the need for a new program. 

In [129]:
%reset -f


In [130]:
import pandas as pd
import numpy as np

data = pd.read_csv('Kinesiology Degrees_10-8-2024 raw data.csv',header=0)

df_raw = pd.DataFrame(data)

df_raw.head()

,UnitID,Institution Name,Grand total (C2023_A First major Exercise Physiology and Kinesiology Bachelor's degree),Grand total (C2023_A First major Exercise Physiology and Kinesiology Master's degree),Grand total (C2023_A First major Exercise Physiology and Kinesiology Doctor's degree - research/scholarship ),Grand total (C2023_A First major Exercise Physiology and Kinesiology Doctor's degree - professional practice ),Grand total (C2023_A First major Exercise Science and Kinesiology Bachelor's degree),Grand total (C2023_A First major Exercise Science and Kinesiology Master's degree),Grand total (C2023_A First major Exercise Science and Kinesiology Doctor's degree - research/scholarship ),Grand total (C2023_A First major Exercise Science and Kinesiology Doctor's degree - professional practice ),...,Grand total (C2020_A_RV Second major Exercise Science and Kinesiology Doctor's degree - research/scholarship ),Grand total (C2020_A_RV Second major Exercise Science and Kinesiology Doctor's degree - professional practice ),Bachelor's degree - programs offered (C2020DEP_RV Exercise Physiology and Kinesiology),Master's degree - programs offered (C2020DEP_RV Exercise Physiology and Kinesiology),Doctor's degree-research/scholarship - programs offered (C2020DEP_RV Exercise Physiology and Kinesiology),Doctor's degree-professional practice - programs offered (C2020DEP_RV Exercise Physiology and Kinesiology),Bachelor's degree - programs offered (C2020DEP_RV Exercise Science and Kinesiology),Master's degree - programs offered (C2020DEP_RV Exercise Science and Kinesiology),Doctor's degree-research/scholarship - programs offered (C2020DEP_RV Exercise Science and Kinesiology),Doctor's degree-professional practice - programs offered (C2020DEP_RV Exercise Science and Kinesiology)
0,493868,A Better U Beauty Barber Academy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,177834,A T Still University of Health Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,180203,Aaniiih Nakoda College,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,491464,ABC Adult School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,459523,ABC Beauty Academy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# Add column totals

df_raw_sums = df_raw.iloc[:,2:].sum()
# Create a new row with 'total' in the first column
new_row = pd.Series(['university name','total'] + df_raw_sums.tolist(), index=df_raw.columns)
# Append the new row to the DataFrame
df_raw = pd.concat([new_row.to_frame().T,df_raw, new_row.to_frame().T], ignore_index=True)

In [132]:
import pandas as pd

#Rename Degrees conferred columns
def rename_deg_columns(col_name):
    # Extract degree level, year, and major from the original column name
    parts = col_name.split('(C')

    year = parts[1][0:4] # Extracts the year (e.g., '2023')
    
    parts2 = col_name.split('_A')
    parts2 = [part.replace('_RV', '') for part in parts2]
    new_name = parts2[1][1:-1] + ' ' + year
    return new_name

#Rename program columns
def rename_prog_columns(col_name):


    parts = col_name.split('DEP')
    parts = [part.replace('_RV', '') for part in parts]
    # print(parts)
    
    parts2 = col_name.split('(C')
    year = parts2[1][0:4] # Extracts the year (e.g., '2023')

    # Extract degree level, year, and major from the original column name
    parts3 = col_name.split()


    new_name = 'Program Counts ' + parts3[0] + ' ' + parts3[1] + ' ' + year + parts[1][1:-1]
    # print(new_name)
    return new_name

# Apply the renaming function to the DataFrame columns
df_raw.columns = [rename_deg_columns(col) if 'major' in col else col for col in df_raw.columns]
df_raw.columns = [rename_prog_columns(col) if 'program' in col else col for col in df_raw.columns]

# df_raw.head()


In [133]:
# Split by CIP code
df_26_0908 = df_raw.loc[:, df_raw.columns.str.contains('Exercise Physiology and Kinesiology|Institution Name')]
df_31_0505 = df_raw.loc[:, df_raw.columns.str.contains('Exercise Science and Kinesiology|Institution Name')]

# Split by program or degree type
df_26_0908_degree = df_26_0908.loc[:, df_26_0908.columns.str.contains('major|Institution Name')]
df_26_0908_programs = df_26_0908.loc[:, df_26_0908.columns.str.contains('Program|Institution Name')]
df_31_0505_degree = df_31_0505.loc[:, df_31_0505.columns.str.contains('major|Institution Name')]
df_31_0505_programs = df_31_0505.loc[:, df_31_0505.columns.str.contains('Program|Institution Name')]


In [134]:
def filter_columns(df, keywords):
    """Filter columns based on the presence of any keyword in the column name."""
    return df[[col for col in df.columns if any(keyword in col for keyword in keywords)]]

In [135]:
# Define years and DataFrame names
years = ['2020', '2021', '2022', '2023']
dataframe_names = [
    'df_26_0908_degree',
    'df_26_0908_programs',
    'df_31_0505_degree',
    'df_31_0505_programs',
]

# Dictionary to hold the split DataFrames
split_dataframes = {}

# Loop through each DataFrame name and year to create new DataFrames
for df_name in dataframe_names:
    for year in years:
        split_name = f"{df_name}_{year}"
        split_dataframes[split_name] = filter_columns(eval(df_name), [year, 'Institution Name'])



In [136]:
# Example access
# split_dataframes['df_26_0908_degree_2020']


In [137]:
split_dataframes.items()

dict_items([('df_26_0908_degree_2020',                                 Institution Name  \
0                                          total   
1               A Better U Beauty Barber Academy   
2        A T Still University of Health Sciences   
3                         Aaniiih Nakoda College   
4                               ABC Adult School   
...                                          ...   
6131                         Zen Shiatsu Chicago   
6132                        Zion Massage College   
6133                             ZMS The Academy   
6134  Zorganics Institute of Beauty and Wellness   
6135                                       total   

      First major  Exercise Physiology and Kinesiology  Bachelor's degree 2020  \
0                                                3744.0                          
1                                                   NaN                          
2                                                   NaN                          
3       

In [138]:
# Drop columns and rows that have no entries
for name, df in split_dataframes.items():
    # Drop columns where the sum is 0
    df = df.loc[:, df.sum() != 0]
    # Drop rows where the sum is 0
    df = df.loc[df.iloc[:,1:].sum(axis=1) != 0]
    # Update the DataFrame in the dictionary
    split_dataframes[name] = df


In [139]:
# Define the output file names
output_files = {
    'df_26_0908_degree': '26_0908_degrees awarded.xlsx',
    'df_26_0908_programs': '26_0908_program counts.xlsx',
    'df_31_0505_degree': '31_0505_degrees awarded.xlsx',
    'df_31_0505_programs': '31_0505_program counts.xlsx',
}

# Create an Excel writer object for each DataFrame
for df_name, output_file in output_files.items():
    with pd.ExcelWriter(output_file) as writer:
        # Iterate through each year to write the corresponding DataFrame
        for year in ['2020', '2021', '2022', '2023']:
            sheet_name = year
            # Check if the DataFrame for that year exists in split_dataframes
            if f"{df_name}_{year}" in split_dataframes:
                split_dataframes[f"{df_name}_{year}"].to_excel(writer, sheet_name=sheet_name, index=False)

# All DataFrames written to their respective Excel files